<a href="https://colab.research.google.com/github/AkankshaGiliyal/binaryClassifier/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers datasets torch scikit-learn
import re
from transformers import DistilBertTokenizerFast
from datasets import Dataset
import pandas as pd


train_df = pd.read_csv("train_enc.tsv", sep="\t", header=None, names=["label", "text"])
dev_df = pd.read_csv("dev_enc.tsv", sep="\t", header=None, names=["label", "text"])


train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)


tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
dev_dataset = dev_dataset.map(tokenize_function, batched=True)
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments,DistilBertModel
from sklearn.metrics import accuracy_score
from transformers import EarlyStoppingCallback
import torch
import torch.nn as nn

class DistilBertLSTM(nn.Module):
    def __init__(self):
        super(DistilBertLSTM, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.lstm = nn.LSTM(input_size=self.distilbert.config.hidden_size, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, 2)

    def forward(self, input_ids, attention_mask, labels=None):

        distilbert_output = self.distilbert(input_ids, attention_mask=attention_mask)
        hidden_states = distilbert_output.last_hidden_state


        lstm_out, (h_n, c_n) = self.lstm(hidden_states)
        last_hidden_state = h_n[-1]


        logits = self.fc(last_hidden_state)


        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))
            return loss, logits
        else:
            return logits


model = DistilBertLSTM()


def compute_metrics(p):
    preds = p.predictions.argmax(axis=1)
    acc = accuracy_score(p.label_ids, preds)
    print(f"Evaluation Accuracy: {acc:.4f}")
    return {"accuracy": acc}


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    report_to=["none"],
    gradient_accumulation_steps=4,
    warmup_steps=500,

)
early_stopping = EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)


trainer.train()
test_df = pd.read_csv("test_enc_unlabeled.tsv", sep="\t", header=None, names=["text"])
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_function, batched=True)

test_predictions = trainer.predict(test_dataset).predictions.argmax(axis=1)


with open("upload_predictions.txt", "w") as f:
    for pred in test_predictions:
        f.write(f"{pred}\n")

print("Predictions saved to upload_predictions.txt")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.674933,0.585594
2,0.641200,0.520872,0.747410
3,0.641200,0.426912,0.810557
4,0.404600,0.393281,0.844598
5,0.404600,0.387518,0.860878
6,0.210000,0.417954,0.862358
7,0.210000,0.399408,0.876665
8,0.120500,0.469088,0.875678
9,0.120500,0.484594,0.873705


Evaluation Accuracy: 0.5856
Evaluation Accuracy: 0.7474
Evaluation Accuracy: 0.8106
Evaluation Accuracy: 0.8446
Evaluation Accuracy: 0.8609
Evaluation Accuracy: 0.8624
Evaluation Accuracy: 0.8767
Evaluation Accuracy: 0.8757
Evaluation Accuracy: 0.8737


Map:   0%|          | 0/2028 [00:00<?, ? examples/s]

Predictions saved to upload_predictions.txt
